In [7]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

### Read in dummy Data

In [3]:
data = pd.read_csv('database.csv')

In [74]:
# data.columns

Index(['Unnamed: 0', 'account_id', 'trans_id', 'category', 'payee', 'k-symbol',
       'date', 'amount', 'mean_income', 'Initial Balance'],
      dtype='object')

In [4]:
cx = pd.read_csv('customers.csv')

In [5]:
print([len(set(cx.account_id)), len(set(data.payee))])
print(max(data.payee))

[4500, 1404]
1403


In [83]:
# Remove inc
#data = data[data.amount > 0]
# data = data.sort_values(['account_id'])
# acct_lkp = {'account_id': data.account_id, 'id': range(cx.shape[0])}
#data

### Data manipulation

In [17]:
# [for each customer, create a row detailing counts for each transaction type]
# e.g.
#
# ID   Type1 Type2 Type3 Type4 Type5
# ====================================
# 001    0     1     0     3     1
# 002    0     0     1     1     0
# 003    1     0     0     2     0
# ...   ..    ..    ..    ..    ..

tfData = np.zeros([cx.shape[0]+1, max(data.payee)+1])
cxTot  = data.shape[0]
cxNum  = 0
for ii in range(data.shape[0]-1):
    tfData[cxNum, data.payee[ii]] += 1
    cxNum += data.account_id[ii+1] != data.account_id[ii]

# do final customer
# cII = ii;
# for ii in range(cII, data.shape[0]-1):
#     tfData[cxNum, data.payee[ii]] += 1
tfData

# Apologies, this is SLOW and inelegant: only used once in hackathon

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [20]:
tfData.shape

238.0

### Test scikit-learn LDA
(Turns out that this is **much** faster than R's current package implementation (not sure if R uses stochastic/minibatch variational EM))

In [12]:
# Inspect example dataset to understand type of 
# ===> CHECK 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
tf
# ===> IE. want sparse matrix where every row is a customer, every column is a transaction type.

In [ ]:
lda = LatentDirichletAllocation(n_topics=25, 
                                max_iter=5, learning_method='online', 
                                learning_offset=50.,random_state=0, verbose=True, n_jobs=-1).fit(tfData)

In [102]:
file1 = open("payees.txt", "r")
payees = file1.readlines()
file1.close()
file2 = open("types.txt", "r")
types = file2.readlines()
file2.close()

In [107]:
num_top_trans = 10
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([payees[i]
                    for i in topic.argsort()[:-num_top_trans - 1:-1]]))

Topic 0:
VIRGINTRAINSEC SERVCS
 DAVID LLOYD00000000
 PREMIER TRADING UK
 RSS ABERGAVENNY
 GOVERNMENT KNOWLEGDE E
 SP  SHERWOOD DIA
 PC WORLD
 COUNCIL TAX
 FUN PARTY LTD
 J D SPORTS  100

Topic 1:
EMAP LTD-SUBSCRIPTIONS
 EASYJET     ENRP
 TFL CC/LEZ CHARGE
 GLOTECH
 FIRSTCHOICE CATERIN
 THE CHALLENGE NETW
 WWW.ADDISONLEE.COM
 TYPOFONDERIE
 UPT CHESHIRE EAST BORO
 MOORE SONS LTD

Topic 2:
WWW.BRITISHNEWSPAPERAC
 WWW.FOUNDATION STAGE
 SUN PLASTICS
 QUBIC PRINT DIRECT
 LONDON PLANT HIRE
 FACEBOOK WLWHA82K52
 OSCARS DEN
 FILMBANK PVSL ACC
 HOTELS MELIA
 RYMAN 1164

Topic 3:
Amazon UK Marketpla
 EA HAZARDOUS WASTE-DOM
 STAPLES UK
 TESCO STORES-2265
 KASHMIR BUTCHERS LTD
 TESCO STORE 2131
 TICKETMASTER UK
 WWW.METROLINE.CO.U
 CURRIE MOTORS CAR
 ADVANTIS CREDIT - MOTO

Topic 4:
PAYPAL NEOTECHNOLO
 FROM CHINA LIMITED
 FLEET CARS & COURIERS
 THOMPSONS OF CREWS
 TSGN000000000000000
 FILMBANK PVSL ACC
 RS LOCKSMITH MLA
 HALFORDS 430
 PHONAK GROUP LTD
 WAITROSE 124

Topic 5:
BOWMONK LTD
 ROYAL MAIL

##### Problems with scikit package:
It is not at all obvious how to extract the categorical distributions by customer from the python LDA implementation: this is a shame as it is extremely fast. Perhaps this can be explored later - Matt Hoffman's script upon which the sklean script is based is clearer, but written in Python 2.x, which I don't currently have installed. Therefore, I've resorted to using R:

### Instead interfact with R's version
(Unfortunately this is extremely slow.)

In [23]:
# R scripts may be called from python via: 
from subprocess import call
call (["/usr/bin/Rscript", "--vanilla", "./r-lda.r"])
    # see script r-lda.R

FileNotFoundError: [Errno 2] No such file or directory: '/usr/bin/Rscript'

In [ ]:
# load inputs in from R
cxDistns     = pd.read_csv('lda_distns.csv')
typeClusters = pd.read_csv('lda_clusters.csv')